In [ ]:
import numpy as np

Description
===========

In this notebook the idea will be to estimate the depth to which we can find the Lyman break in type 1 AGN as a function of L/L* at z=2.5 . For this, we will need to know the depth in u-band and see what luminosity this corresponds to. Then, we can determine the same for the g-band 5 sigma depth so that we can see how much we are missing.

In [ ]:
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles
from lsst.sims.maf.metrics import BaseMetric

import os
import sys
sys.path.append("../../LSST_OpSim/Scripts_NBs/")
from opsimUtils import *

### Step 1

The first step will be to run all the metrics to get the ExgalM5 depths. We will do this for both the *u* and *g* bands. We will base this on a slight modification of the DESC ExgalM5_with_cuts metric of DESC (https://github.com/lsst/sims_maf/blob/master/python/lsst/sims/maf/metrics/weakLensingSystematicsMetric.py).

In [ ]:
from ExgalM5_with_cuts_AGN import ExgalM5_with_cuts_AGN

In [ ]:
#We will use the same slicer for both bands. We use nside=128 to match the nside of the extinction map.
#A warning, otherwise, pops out. Seems harmless, but better use nsider=128 to be safe. Also, we add 
#useCache=False again to deal with another warning.
slicer_ug = slicers.HealpixSlicer(nside=128, useCache=False)

In [ ]:
#Set up the MAF for u-band, 5sigma depth.
metric_u = ExgalM5_with_cuts_AGN(metricName='ExgalM5_with_cuts_AGN_u')

# constraint = the sql query (or 'select') that selects all visits in r band
constraint_u = 'filter = "u"'
constraint_u += ' and note not like "DD%"' # added so the sky plot won't saturate (remove DDFs)

EM5u = metricBundles.MetricBundle(metric_u, slicer_ug, constraint_u)

In [ ]:
#Set up the MAF for g-band, 5sigma depth.
metric_g = ExgalM5_with_cuts_AGN(metricName='ExgalM5_with_cuts_AGN_g')

# constraint = the sql query (or 'select') that selects all visits in r band
constraint_g = 'filter = "g"'
constraint_g += ' and note not like "DD%"' # added so the sky plot won't saturate (remove DDFs)

EM5g = metricBundles.MetricBundle(metric_g, slicer_ug, constraint_g)

In [ ]:
bundleDict = {'EM5u': EM5u, 'EM5g': EM5g}

##### Run for FBS 1.5

In [ ]:
your_username = "rjassef"
folder_mafoutput = "EM5_depths"

In [ ]:
FBS_version = "1.5"
dbDir = '/home/idies/workspace/lsst_cadence/FBS_{}/'.format(FBS_version)
outDir = '/home/idies/workspace/Storage/{0}/persistent/MAFOutput/{1}'.format(your_username,folder_mafoutput)

if not os.path.exists(os.path.abspath(outDir)):
    os.mkdir(os.path.abspath(outDir))

In [ ]:
opSimDbs, resultDbs = connect_dbs(dbDir, outDir)

In [ ]:
metricDataPath = '/home/idies/workspace/Storage/{0}/persistent/MAFOutput/{1}/MetricData/'.format(
    your_username, folder_mafoutput)
dbRuns = show_opsims(dbDir)
for run in dbRuns:
    EM5u.setRunName(run)
    EM5g.setRunName(run)
    metricGroup = metricBundles.MetricBundleGroup(bundleDict,\
                    opSimDbs[run], metricDataPath, resultDbs[run])
    metricGroup.runAll()

##### Run for FBS 1.6

In [ ]:
FBS_version = "1.6"
dbDir = '/home/idies/workspace/lsst_cadence/FBS_{}/'.format(FBS_version)
outDir = '/home/idies/workspace/Storage/{0}/persistent/MAFOutput/{1}'.format(your_username,folder_mafoutput)

if not os.path.exists(os.path.abspath(outDir)):
    os.mkdir(os.path.abspath(outDir))

In [ ]:
opSimDbs, resultDbs = connect_dbs(dbDir, outDir)

In [ ]:
metricDataPath = '/home/idies/workspace/Storage/{0}/persistent/MAFOutput/{1}/MetricData/'.format(
    your_username, folder_mafoutput)
dbRuns = show_opsims(dbDir)
for run in dbRuns:
    EM5u.setRunName(run)
    EM5g.setRunName(run)
    metricGroup = metricBundles.MetricBundleGroup(bundleDict,\
                    opSimDbs[run], metricDataPath, resultDbs[run])
    metricGroup.runAll()

### Step 1

Estimate the u and g magnitude of an Lstar quasar at z=2.5. We do this by using the Shen et al. (2020) luminosity function to calculate Lbol_star at z=2.5. We then use the conversion from Lbol to L1450 provided by Shen et al. (2020). Finally, we take a quasar template, redshift it to z=2.5, normalize it to L1450 (calculated using a redshifted tophat template as described in Shen et al.) and calcute its magnitudes in the SDSS u and g bands. There are two options of templatates so far:

* vanden Berk et al. (2001) - All the manipulation of the vanden Berk et al. template is done using pysynphot, which is not available in SciServer. 

* Assef et al. (2010) - All manipulation is done with LRT, which I have not installed in my SciServer workspace (might actually not be possible).

We assume a Planck13 astropy cosmology in either case. The pre-computed magnitude-redshift tables are found in the folder ../mstar_estimates. 

In [ ]:
#vanden Berk et al. (2001)
data = np.loadtxt("../mstar_estimates/mstar_z.vandenberk.dat")

#Assef et al. (2010)
#data = np.loadtxt("../mstar_estimates/mstar_z.lrt.dat")

mstar = dict()
filters = ['u','g','r','i','z','y']
for k, filter in enumerate(filters):
    mstar[filter] = np.interp([2.5], data[:,0], data[:,k+1])[0]
    print("{0:s} {1:7.3f}".format(filter, mstar[filter]))

### Step 2

To detect the Lyman break, we will need to have a color u-g that has an uncertainty of less that ~0.33 mag (so a 3 sigma detection). Assuming that g is always going to be much deeper than u, then the uncertainty will be dominated by u. In that sense, we want to determine the fraction of L* for which a 3 sigma u-band detection gets us, and the fraction of L* to which the 5 sigma g depth gets us.

We can only get the 5 sigma depth as far as I can see with a metric, so we will simply assume that for a given 5 sigma depth, m5, the corresponding 3 sigma depth will be 

m3 = m5 + 2.5 log10(5/3)

With that in mind, we will create a metric that determines L/L* for a 5 sigma depth, to be used for g band, and one that determines L/L* for a 3 sigma depth, to be used for u band. 

To create the metrics I started from the Coaddm5Metric shown in https://github.com/LSST-nonproject/sims_maf_contrib/blob/0c4c7f9f1fc112c4d91e372ae338b29b054468a4/tutorials/Writing%20A%20New%20Metric.ipynb and the ExgalM5_with_cuts metric from DESC available at https://github.com/lsst/sims_maf/blob/master/python/lsst/sims/maf/metrics/weakLensingSystematicsMetric.py.

In [ ]:
class log_QLF_Lstar_frac(BaseMetric):
    
    def __init__(self, mstar, m5Col='fiveSigmaDepth', sigma_det=5.0, extinction_cut=1.0, \
                 metricName='log_QLF_Lstar_frac', **kwargs):
        
        #Apparent magnitude of an Lstar quasar. Has to be externally provided,
        #as my calculation involves pysynphot, which is not available in SciServer.
        self.mstar = mstar
        
        #Dust Extinction limit to which consider regions. If left unconstrained,
        #it ends up finding extremely shallow (m_lim=100) 5 sigma regions. Not
        #sure why though, but this is something also enforced in the ExgalM5_with_cuts
        #metric of DESC.
        self.extinction_cut = extinction_cut
        
        #dm is the difference between the 5 sigma detection and the requested sigma_det.
        self.dm = 2.5*np.log10(5.0/sigma_det)
        
        #This calculation is reliant on the ExgalM5 metric. So declare that here.
        self.exgalM5 = metrics.ExgalM5()
        
        #Initiate the metric.
        super(log_QLF_Lstar_frac, self).__init__(
            col=m5Col, metricName=metricName, maps=self.exgalM5.maps, **kwargs)
        
    def run(self, dataSlice, slicePoint=None):
        # exclude areas with high extinction
        if slicePoint['ebv'] > self.extinction_cut:
            return self.badval
        
        mlim5 = self.exgalM5.run(dataSlice, slicePoint)
        mlim  = mlim5 + self.dm
        return -0.4*(mlim-self.mstar)

In [ ]:
#We will use the same slicer for both bands.
slicer_ug = slicers.HealpixSlicer(nside=64)

In [ ]:
#Set up the MAF for u-band, 3sigma depth.
metric_u = log_QLF_Lstar_frac(mstar_u, sigma_det=3.0) #logLstarfracCoaddm3_dustMetric(mstar_u)

# constraint = the sql query (or 'select') that selects all visits in r band
constraint_u = 'filter = "u"'
constraint_u += ' and note not like "DD%"' # added so the sky plot won't saturate (remove DDFs)

logLstarfrac_u = metricBundles.MetricBundle(metric_u, slicer_ug, constraint_u)

In [ ]:
#Set up the MAF for u-band, 3sigma depth.
metric_g = log_QLF_Lstar_frac(mstar_g, sigma_det=5.0) #logLstarfracCoaddm5_dustMetric(mstar_g)
constraint_g = 'filter = "g"'
constraint_g += ' and note not like "DD%"' # added so the sky plot won't saturate (remove DDFs)
logLstarfrac_g = metricBundles.MetricBundle(metric_g, slicer_ug, constraint_g)

In [ ]:
bundleDict = {'logLstarfrac_u': logLstarfrac_u, 'logLstarfrac_g': logLstarfrac_g}

### Step 3

Setup the environment to run the metrics on many opsims. For this, we follow the [Multiple_Opsims.ipynb](./Scripts_NBs/Multiple_Opsims.ipynb) example from Gordon's group. 

In [ ]:
import os
from Scripts_NBs.opsimUtils import *

In [ ]:
your_username = "rjassef"
dbDir = '/home/idies/workspace/lsst_cadence/FBS_1.6/'
outDir = '/home/idies/workspace/Storage/{}/persistent/MAFOutput/Lstar_dust'.format(your_username)

if not os.path.exists(os.path.abspath(outDir)):
    os.mkdir(os.path.abspath(outDir))

In [ ]:
opSimDbs, resultDbs = connect_dbs(dbDir, outDir)

### Step 4 

Loop over the opsims running the metric on all of them. 

In [ ]:
metricDataPath = '/home/idies/workspace/Storage/{}/persistent/MAFOutput/Lstar_dust/MetricData/'.format(your_username)
dbRuns = show_opsims(dbDir)
for run in dbRuns:
    logLstarfrac_u.setRunName(run)
    logLstarfrac_g.setRunName(run)
    metricGroup = metricBundles.MetricBundleGroup(bundleDict,\
                    opSimDbs[run], metricDataPath, resultDbs[run])
    metricGroup.runAll()

### Step 5

Repeat for FBS v1.5

In [ ]:
import os
from Scripts_NBs.opsimUtils import *

In [ ]:
your_username = "rjassef"
dbDir = '/home/idies/workspace/lsst_cadence/FBS_1.5/'
outDir = '/home/idies/workspace/Storage/{}/persistent/MAFOutput/Lstar_dust'.format(your_username)

if not os.path.exists(os.path.abspath(outDir)):
    os.mkdir(os.path.abspath(outDir))

In [ ]:
opSimDbs, resultDbs = connect_dbs(dbDir, outDir)

In [ ]:
metricDataPath = '/home/idies/workspace/Storage/{}/persistent/MAFOutput/Lstar_dust/MetricData/'.format(your_username)
dbRuns = show_opsims(dbDir)
for run in dbRuns:
    logLstarfrac_u.setRunName(run)
    logLstarfrac_g.setRunName(run)
    metricGroup = metricBundles.MetricBundleGroup(bundleDict,\
                    opSimDbs[run], metricDataPath, resultDbs[run])
    metricGroup.runAll()